In [ ]:
import cv2
import pyttsx3
import datetime
import speech_recognition as sr
import wikipedia
import webbrowser as wb
import os
import random
import pyautogui
import subprocess
import requests
from bs4 import BeautifulSoup

# Initialize speech engine
engine = pyttsx3.init()

def speak(audio):
    try:
        print("Jarvis:", audio)
        engine.say(audio)
        engine.runAndWait()
    except RuntimeError:
        pass

def wishme():
    hour = datetime.datetime.now().hour
    greet = "Good Morning" if hour < 12 else "Good Afternoon" if hour < 18 else "Good Evening"
    speak(f"{greet} Madam!!")
    speak("Jarvis at your service. Please tell me how may I help you.")

def take_command():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening...")
        r.pause_threshold = 0.5
        audio = r.listen(source, timeout=30, phrase_time_limit=10)
    try:
        print("Recognizing...")
        query = r.recognize_google(audio, language="en-in")
        print(f"You said: {query}")
    except Exception:
        speak("Please say that again")
        return "None"
    return query.lower()

# --- Environment Functions ---
def browser_task(query):
    if "top ai headlines" in query or "ai headlines" in query:
        speak("Fetching top AI headlines.")
        try:
            url = "https://www.bing.com/news/search?q=artificial+intelligence"
            page = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
            soup = BeautifulSoup(page.content, 'html.parser')
            headlines = [a.get_text(strip=True) for a in soup.select("a.title")[:5]]
            if headlines:
                for i, headline in enumerate(headlines, 1):
                    speak(f"Headline {i}: {headline}")
            return headlines
        except Exception as e:
            speak("Failed to fetch headlines")
            return []

def terminal_task(headlines):
    try:
        formatted = "Top 5 AI News Headlines:\n\n" + "\n".join([f"{i+1}. {h}" for i, h in enumerate(headlines)])
        return formatted
    except Exception:
        return "Error formatting headlines."

def file_task(content, filename="AI_Headlines_Report.txt"):
    try:
        path = os.path.expanduser(f"~\Documents\{filename}")
        with open(path, "w", encoding='utf-8') as file:
            file.write(content)
        speak("Headlines saved to file successfully.")
    except Exception as e:
        speak("Error saving the file.")

def face_detection():
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    cap = cv2.VideoCapture(0)

    detected = False
    speak("Starting face detection...")
    while True:
        ret, frame = cap.read()
        if not ret:
            continue
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x,y), (x+w, y+h), (255,0,0), 2)
            detected = True
            break

        cv2.imshow("Face Detection - Press Q to continue", frame)
        if cv2.waitKey(1) & 0xFF == ord('q') or detected:
            break

    cap.release()
    cv2.destroyAllWindows()
    if detected:
        speak("Face recognized.")
    else:
        speak("No face detected. Exiting.")
        exit()

# --- Main Assistant Logic ---
def jarvis():
    wishme()
    while True:
        query = take_command()

        if "time" in query:
            current_time = datetime.datetime.now().strftime("%I:%M %p")
            speak(f"The current time is {current_time}")

        elif "date" in query:
            today = datetime.date.today()
            speak(f"The current date is {today.strftime('%d/%m/%Y')}")

        elif "who are you" in query:
            speak("I'm JARVIS created by Ms. Bhavani and I'm a desktop voice assistant.")

        elif "how are you" in query:
            speak("I'm fine Madam, What about you?")

        elif any(word in query for word in ["fine", "good"]):
            speak("Glad to hear that Ma'am!!")

        elif "wikipedia" in query:
            speak("Searching Wikipedia...")
            try:
                query = query.replace("wikipedia", "")
                result = wikipedia.summary(query, sentences=2)
                speak(result)
            except:
                speak("Couldn't find the page.")

        elif "open youtube" in query:
            wb.open("https://youtube.com")

        elif "open google" in query:
            wb.open("https://google.com")

        elif "open stack overflow" in query:
            wb.open("https://stackoverflow.com")

        elif "play music" in query:
            music_dir = os.path.expanduser("~\Music")
            songs = os.listdir(music_dir)
            if songs:
                song = random.choice(songs)
                path = os.path.join(music_dir, song)
                wmplayer = r"C:\Program Files (x86)\Windows Media Player\wmplayer.exe"
                subprocess.call([wmplayer, path])
            else:
                speak("No music files found.")

        elif "open chrome" in query:
            chrome = r"C:\Program Files\Google\Chrome\Application\chrome.exe"
            os.startfile(chrome)

        elif "search on chrome" in query or "search for" in query:
            if "search on chrome" in query:
                speak("What should I search?")
                search = take_command()
            else:
                search = query.replace("search for", "").strip()
            if search != "None" and search != "":
                wb.open(f"https://www.google.com/search?q={search}")

        elif "remember that" in query:
            data = query.replace("remember that", "").strip()
            with open("data.txt", "w") as f:
                f.write(data)
            speak(f"You asked me to remember that: {data}")

        elif "do you remember anything" in query:
            try:
                with open("data.txt", "r") as f:
                    remembered = f.read()
                speak(f"You asked me to remember that {remembered}")
            except:
                speak("I don't remember anything yet.")

        elif "screenshot" in query:
            img = pyautogui.screenshot()
            timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
            path = os.path.expanduser(f"")
            img.save(path)
            speak("Screenshot taken.")

        elif "ai headlines" in query:
            headlines = browser_task(query)
            if headlines:
                report = terminal_task(headlines)
                file_task(report)

        elif "offline" in query:
            speak("Goodbye Madam!")
            break

        else:
            speak("Sorry, I didn't catch that. Please try again.")

if __name__ == "__main__":
    face_detection()
    jarvis()


Jarvis: Starting face detection...
Jarvis: Face recognized.
Jarvis: Good Evening Madam!!
Jarvis: Jarvis at your service. Please tell me how may I help you.
Listening...
Recognizing...
You said: remember that
Jarvis: You asked me to remember that: 
Listening...
Recognizing...
You said: Virat Kohli
Jarvis: Sorry, I didn't catch that. Please try again.
Listening...
Recognizing...
